In [1]:

import sys
from os.path import join
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array
#from tensorflow.python.keras.applications import ResNet50

from keras import models, regularizers, layers, optimizers, losses, metrics
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils, to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.applications import ResNet50
import keras
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import tensorflow

import os

Using TensorFlow backend.


In [2]:

# Convoluted Base MODEL

conv_base = ResNet50(weights='imagenet',include_top=False, input_shape=(224, 224, 3))

print(conv_base.summary())


C:\Users\BioHelwan\Anaconda3\envs\tensorflow\Lib\site-packages\keras_applications\resnet50.py:263: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [3]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(64, activation='relu',kernel_regularizer=regularizers.l2(0.001)))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(5, activation='sigmoid'))

print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 100352)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                6422592   
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 325       
Total params: 30,010,629
Trainable params: 29,957,509
Non-trainable params: 53,120
___________________________________________________________

In [4]:

for layer in conv_base.layers[:]:
   layer.trainable = False

print('conv_base is now NOT trainable')


for i, layer in enumerate(conv_base.layers):
   print(i, layer.name, layer.trainable)
   
   
   
model.compile(optimizer=optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("model compiled")
print(model.summary())

conv_base is now NOT trainable
0 input_1 False
1 conv1_pad False
2 conv1 False
3 bn_conv1 False
4 activation_1 False
5 max_pooling2d_1 False
6 res2a_branch2a False
7 bn2a_branch2a False
8 activation_2 False
9 res2a_branch2b False
10 bn2a_branch2b False
11 activation_3 False
12 res2a_branch2c False
13 res2a_branch1 False
14 bn2a_branch2c False
15 bn2a_branch1 False
16 add_1 False
17 activation_4 False
18 res2b_branch2a False
19 bn2b_branch2a False
20 activation_5 False
21 res2b_branch2b False
22 bn2b_branch2b False
23 activation_6 False
24 res2b_branch2c False
25 bn2b_branch2c False
26 add_2 False
27 activation_7 False
28 res2c_branch2a False
29 bn2c_branch2a False
30 activation_8 False
31 res2c_branch2b False
32 bn2c_branch2b False
33 activation_9 False
34 res2c_branch2c False
35 bn2c_branch2c False
36 add_3 False
37 activation_10 False
38 res3a_branch2a False
39 bn3a_branch2a False
40 activation_11 False
41 res3a_branch2b False
42 bn3a_branch2b False
43 activation_12 False
44 res3a_br

In [5]:
train_dir = 'D:/Breast Cancer Grad.Proj/roi/train'
test_dir = 'D:/Breast Cancer Grad.Proj/roi/test'

In [6]:
data_generator = ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255
)


datagen = ImageDataGenerator(
    preprocessing_function= \
    tensorflow.keras.applications.resnet50.preprocess_input)

train_batches = ImageDataGenerator().flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'], batch_size=64)
validation_batches = ImageDataGenerator().flow_from_directory(train_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'], batch_size=64)
test_batches = ImageDataGenerator().flow_from_directory(test_dir, target_size=(224,224),
    classes=['benign_calcification','benign_mass','malignant_mass','malignant_calcification','nothing'], batch_size=10)


Found 25999 images belonging to 5 classes.
Found 25999 images belonging to 5 classes.
Found 1344 images belonging to 5 classes.


In [7]:


train_steps = np.ceil(25999/64)
validation_steps = np.ceil(25999/64)



In [8]:
model.compile(optimizer=Adam(lr=.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
filepath="D:/Breast Cancer Grad.Proj/weights-improvement-{epoch:02d}-{val_acc:.2f}.hdf5"

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint( filepath, save_weights_only=True,  monitor='val_acc', mode='max', save_best_only=True)


In [ ]:

model.fit_generator(generator=train_batches, steps_per_epoch=train_steps, 
                    validation_data=validation_batches, validation_steps=validation_steps, epochs=100, callbacks=[model_checkpoint_callback], verbose=2)





Epoch 1/100
 - 870s - loss: 1.3479 - acc: 0.4629 - val_loss: 1.0836 - val_acc: 0.6115
Epoch 2/100
 - 649s - loss: 1.1573 - acc: 0.5512 - val_loss: 0.9581 - val_acc: 0.6454
Epoch 3/100
 - 648s - loss: 1.0922 - acc: 0.5768 - val_loss: 0.9158 - val_acc: 0.6590
Epoch 4/100
 - 647s - loss: 1.0420 - acc: 0.6072 - val_loss: 0.8561 - val_acc: 0.6882
Epoch 5/100
 - 648s - loss: 1.0085 - acc: 0.6263 - val_loss: 0.8289 - val_acc: 0.6836
Epoch 6/100
 - 648s - loss: 0.9712 - acc: 0.6409 - val_loss: 0.7955 - val_acc: 0.7136
Epoch 7/100
 - 648s - loss: 0.9470 - acc: 0.6551 - val_loss: 0.7799 - val_acc: 0.7299
Epoch 8/100
 - 648s - loss: 0.9254 - acc: 0.6645 - val_loss: 0.7107 - val_acc: 0.7518
Epoch 9/100
 - 649s - loss: 0.8992 - acc: 0.6770 - val_loss: 0.7179 - val_acc: 0.7604
Epoch 10/100
 - 648s - loss: 0.8865 - acc: 0.6810 - val_loss: 0.6899 - val_acc: 0.7699
Epoch 11/100
 - 648s - loss: 0.8733 - acc: 0.6927 - val_loss: 0.6797 - val_acc: 0.7900
Epoch 12/100
 - 648s - loss: 0.8605 - acc: 0.7015 - 